In [3]:
from CatanEnv import CatanSetupEnv
from sb3_contrib.ppo_mask import MaskablePPO

env = CatanSetupEnv()

model = MaskablePPO("MlpPolicy", env, gamma=0.4, verbose=1) #, tensorboard_log="./tensorboard_logs/")
# model.learn(total_timesteps=50_000) #, tb_log_name='simple_setup_phase_training')

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [7]:
from sb3_contrib.common.maskable.utils import get_action_masks
from GameStateViewer import SaveGameStateImage
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from ModelState import getInputState

##############################################################
rewardList = []
rewardList100 = []
winner = [0,0,0,0]

for episode in range(1):
    done = False
    state, info = env.reset()

    while done != True:
        action_masks = get_action_masks(env)
        action, _states = model.predict(state, action_masks=action_masks)
        state, reward, done, _, info = env.step(action.item())
        # print(f"Reward: {reward+7}")
        rewardList.append(reward)
    
    # Once initial placement is done use random agents till game is over
    while True:
        currPlayer = env.game.gameState.players[env.game.gameState.currPlayer]

        agentAction = currPlayer.DoMove(env.game)
        agentAction.ApplyAction(env.game.gameState)
        print(len(getInputState(env.game.gameState)))

        if env.game.gameState.currState == "OVER":
            break
    winner[env.game.gameState.winner] += 1


print(winner)



# plt.plot(rewardList)
# plt.xlabel('Episode (100)')
# plt.ylabel('Avg VP')
# plt.show()

# img = mpimg.imread('TRAINING.png')
# plt.imshow(img)
# plt.axis('off')  # Optional: Remove axes for a clean display
# plt.show()

In [ ]:
###################################################################################################################################################

In [6]:
from CatanEnv import CatanSetupEnv, CatanEnv
from sb3_contrib.ppo_mask import MaskablePPO

env = CatanEnv()

model = MaskablePPO("MlpPolicy", env, gamma=0.4, verbose=1, tensorboard_log="./tensorboard_logs/")
model.learn(total_timesteps=500_000, tb_log_name='full_training')

In [5]:
from sb3_contrib.common.maskable.utils import get_action_masks
from GameStateViewer import SaveGameStateImage
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from ModelState import getInputState

##############################################################
rewardList = []
rewardList100 = []
winner = [0,0,0,0]
vpList = []

for episode in range(100):
    done = False
    state, info = env.reset()

    while done != True:
        action_masks = get_action_masks(env)
        action, _states = model.predict(state, action_masks=action_masks)
        state, reward, done, _, info = env.step(action.item())
        # print(f"Reward: {reward+7}")
        rewardList.append(reward)
    
    winner[env.game.gameState.winner] += 1
    vpList.append(env.players[2].victoryPoints)


print(winner)
print(sum(vpList)/len(vpList))

[27, 26, 15, 32]
5.37


In [ ]:
###################################################################################################################################################

In [7]:
from DeepLearning.Env import CatanSetupRandomEnv
from sb3_contrib.ppo_mask import MaskablePPO
import os
os.environ['SAVE_IMAGE'] = 'False'

env = CatanSetupRandomEnv()

model = MaskablePPO("MlpPolicy", env, gamma=0.4, verbose=1, tensorboard_log="./tensorboard_logs/")
model.learn(total_timesteps=100_000, tb_log_name='ENV-SetupRandom_REWARD-win')

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/Users/danieldrummond/Catan/PyCatron/TC2/Client/env/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.action_masks to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.action_masks` for environment variables or `env.get_wrapper_attr('action_masks')` that will search the reminding wrappers.
  logger.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 2        |
|    ep_rew_mean     | 0        |
| time/              |          |
|    fps             | 202      |
|    iterations      | 1        |
|    time_elapsed    | 10       |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2           |
|    ep_rew_mean          | 0.16        |
| time/                   |             |
|    fps                  | 172         |
|    iterations           | 2           |
|    time_elapsed         | 23          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.012719718 |
|    clip_fraction        | 0.195       |
|    clip_range           | 0.2         |
|    entropy_loss         | -3.71       |
|    explained_variance   | -0.119      |
|    learning_rate        | 0.

In [10]:
from sb3_contrib.common.maskable.utils import get_action_masks
from DeepLearning.Env import CatanSetupRandomEnv
import os
from Game.CatanPlayer import PlayerStatsTracker

# os.environ['SAVE_IMAGE'] = 'True'
env = CatanSetupRandomEnv()

rewardList = []
rewardList100 = []
winner = [0,0,0,0]
vpList = []

stats = PlayerStatsTracker()

for episode in range(100):
    done = False
    state, info = env.reset()

    while done != True:
        action_masks = get_action_masks(env)
        action, _states = model.predict(state, action_masks=action_masks)
        state, reward, done, _, info = env.step(action.item())
        # print(f"Reward: {reward+7}")
        rewardList.append(reward)

    
    winner[env.game.gameState.winner] += 1
    
    env.game.gameState.players[0].generatePlayerStats()
    # print(env.game.gameState.players[0].stats)
    stats += env.game.gameState.players[0].stats

stats.getAverages()
print(stats)

print("Winnings: ", winner)

General
 numTurns: 68.88
 victoryPoints: 8.07
 finalTradeRates: [3.59, 3.6, 3.58, 3.62, 3.66]
Dev Card Breakdown
 devCardsBought: 4.97
 usedDevCards: [2.78, 0.27, 0.4, 0.45, 0.0]
Point Breakdown
 settlementsBuilt: 2.09
 citiesBuilt: 1.73
 devCardVP: 0.9
 largestArmy: 0.39
 longestRoad: 0.39
Resource Breakdown
 resourcesReceived: [35.67, 46.48, 36.78, 50.87, 40.33]
 totalResourcesDiscarded: 49.18
 totalResourcesStolen: 12.65
 resourcesFromDevCard: [0.75, 0.59, 0.31, 0.17, 0.16]
 resourcesFromBankTrade: [7.53, 7.54, 7.71, 7.36, 8.06]
 finalResourceProduction: [6.91, 10.29, 8.5, 10.56, 8.87]
Setup Breakdown
 setupResourceProduction: [3.55, 4.23, 3.43, 4.76, 3.8]
 setupTradeRates: [3.92, 3.96, 3.9, 3.96, 3.96]

Winnings:  [48, 20, 17, 15]
